In [1]:
# import openai
# from openai import AzureOpenAI
# import os 
# from azure.identity import ManagedIdentityCredential

# default_credential=ManagedIdentityCredential(client_id="XXX")
# token=default_credential.get_token("https://cognitiveservices.azure.com/.default")
# Resource_endpoint="XXX"

# client = AzureOpenAI(
#   azure_endpoint = Resource_endpoint, 
#   api_key=token.token,  
#   api_version="2023-05-15"
# )

In [103]:
#pip install python-dotenv

In [104]:
#pip install openai

In [105]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-01-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [2]:
#print(os.getenv("AZURE_OPENAI_ENDPOINT"))

## 1. Understand the AOAI Models' capabilities. Start with the latest model, prove your idea , then test with smaller models. 
Model size is critical for better performance.

## 2. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

** Length ** control (specify desired output length e.g.: number of words)

** Tone ** control (e.g.: polite, passionate, professional, technical, funny, casual, serious etc.)

** Style ** control (e.g.: in the style of Shakespeare, JK Rowling, Nelson Mandela etc.)

** Audience ** control (e.g.: a 5-year-old can understand etc)

** Context ** control (e.g.: news, novel, textbook, report, white paper, blog etc.)

In [109]:
base_system_message = "You are a helpful assistant."
system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = " Write a 2 paragraph inspiring poem which can be understood by 5 years-old child focussing on pensions."
# Write a 2 paragraph inspiring poem about Iceland the food warehouse
# Write a 2 paragraph inspiring poem focussing on products of Iceland the food warehouse in a funny way
# Write a 2 paragraph inspiring poem focussing on products of Iceland the food warehouse in the style of Shakespeare

In [110]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

Sure! Here's a simple and inspiring two-paragraph poem about pensions that a five-year-old could understand:

---

When you grow up, and work each day,  
You save some pennies along the way.  
Like planting seeds in a tiny pot,  
They’ll grow and grow—don’t need a lot!  

And when you're old, with hair of gray,  
Your saved-up pennies will brighten your day.  
They'll help you smile, relax, and rest,  
Because you planned—and that's the best!  

--- 

This poem simplifies the concept of pensions, making it relatable to a child by using imagery of saving pennies and planting seeds.


## 3. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [111]:
text = f"""
We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n \
In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
broadly sharing benefits and the need to prioritize safety. \
Microsoft shares this vision and our values, and our partnership is instrumental to our progress.
"""

In [112]:
prompt = f"""
Summarize the text delimited by hashtags as a bullet point list of the most important points.
###{text}###
"""

In [113]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0,
    max_tokens=60
)

# Print the result
print(response.choices[0].message.content.strip())


- OpenAI and Microsoft are extending their partnership with a multi-year, multi-billion dollar investment.  
- This investment builds on Microsoft's previous investments in 2019 and 2021.  
- The funding will support OpenAI's independent research and the development of safe, useful, and powerful AI.


## 4. Articulate the desired output format through examples

In [114]:
system_message = "You are a helpful assistant."

In [115]:
user_message=f"""Extract the entities mentioned in the text below. 
First extract all company names, then extract all years, 
then extract specific topics which fit the content and finally extract general overarching themes\n\n 
Desired format: 
Company names: <comma_separated_list_of_company_names> 
Years: 
Specific topics:
General themes: 
### Text:
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n\n 
###
"""


In [116]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: AI safety, AI research, AI development, partnership, investment  
General themes: Artificial intelligence, technology collaboration, innovation, corporate investment  


## 5.Start with zero-shot, then few-shot (example)

In [117]:
system_message = "You are a helpful assistant."

In [118]:
prompt_zero=f"""Extract most important keywords from the corresponding texts below.\n\n 

###Text: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords:###
"""

In [119]:
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_zero}
    ]
)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, 2019, 2021, independent research, AI, safe, useful, powerful.


In [120]:
prompt_few=f"""Extract most important keywords from the corresponding texts below.\n\n 
### Text 1: 
Stripe provides APIs that web developers can use to integrate 
payment processing into their websites and mobile applications. \n
Keywords 1: Stripe, payment processing, APIs, web developers, websites 
### 

###Text 2: 
OpenAI has trained cutting-edge language models that are very good at understanding 
and generating text. Our API provides access to these models and can be used to solve virtually 
any task that involves processing language. \n
Keywords 2: OpenAI, language models, text processing, API.
### 

###Text 3: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords 3:"""

In [121]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_few}
    ]
)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, investment, AI research, safe AI, useful AI, powerful AI.


## 6.Instead of just saying what not to do, say what to do instead

In [122]:
system_message= f"""You are an agent trying to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. 
Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n"""

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I can’t log in to my account."

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [123]:
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

If you're unable to log into your account, there could be a few potential reasons for the issue. Here’s how you can troubleshoot:

1. **Check Your Credentials**:
   - Ensure that you're entering the correct email address or username (as applicable) and password.
   - Verify that the Caps Lock key is not accidentally on, as passwords are case-sensitive.

2. **Reset Your Password**:
   - Use the "Forgot Password" option on the login page to reset your password. Instructions will be sent to your registered email address.

3. **Clear Cache and Cookies**:
   - Sometimes, browser cache or cookies can cause login issues. Clear them and try again.

4. **Try a Different Browser or Device**:
   - Logging in using a different web browser or device can help identify if the issue is device-specific.

5. **Check for Account Lock or Suspension**:
   - If you've attempted to log in multiple times with incorrect details, your account might be temporarily locked. Wait for a while before trying again.

F

## 7. Divide complex tasks into sub-tasks

In [59]:
system_message = "You are a helpful assistant."

In [124]:
text = f"""
As an FSI company, we want our pension schemes to have a positive impact on our customers. Whether you're just starting to save into a pension or ready to take money out of it, we have the best interests of our members and customers at heart. It's about long-term financial wellbeing and sharing responsibility for building a better future. Save today to enjoy tomorrow."""
# example 1
user_message = f"""
Perform the actions below by separating your answers with line breaks. 
1 - Summarize the following text below with 1 sentence in English.
2 - Translate the summary into Turkish.
3 - List each company name in the Turkish summary.
4 - Output a json object that contains the following:
keys: turkish_summary, turkish_company_names.

###
Text:
{text} 
###
"""

In [125]:
response = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

1 - The FSI company aims to provide positive impact on customers through their pension schemes. 
2 - FSI şirketi, müşterilerine emeklilik planları aracılığıyla pozitif etki sağlamayı hedefliyor.
3 - FSI
4 - {
  "turkish_summary": "FSI şirketi, müşterilerine emeklilik planları aracılığıyla pozitif etki sağlamayı hedefliyor.",
  "turkish_company_names": ["FSI"]
}


## 8. Chain of Thought

The language model is prompted to generate a few intermediate reasoning steps to arrive at the final answer. 

Uses "greedy decoding" which means selecting the most likely token (word or character) at each step of the sequence generation process. At each time step, the model predicts the next token based on the previously generated tokens, and the token with the highest predicted probability is chosen as the output for that step. This process is repeated until the desired sequence length is reached or until a special end-of-sequence token is generated.

**Temp=0** is used because it uses greedy decoding. It first creates the greedy coding and then the answer.

In [136]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: The answer (arabic numerals) is
"""

In [138]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": PROMPT_ZERO_SHOT}
    ],
    temperature=0,
    max_tokens=100,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


Roger starts with 5 tennis balls. He buys 2 cans, each containing 3 tennis balls, so he gets \( 2 \times 3 = 6 \) tennis balls from the cans. Adding these to the 5 he already has, he now has \( 5 + 6 = 11 \) tennis balls. After giving 4 to his friend, he is left with \( 11 - 4 = 7 \) tennis balls.

The answer is **7


In [133]:
PROMPT_ZERO_SHOT_CoT = """Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Let’s think step by step.
"""

In [135]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": PROMPT_ZERO_SHOT_CoT}
    ],
    temperature=0,
    max_tokens=300,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


Sure! Let's break it down step by step:

1. **Initial tennis balls Roger has**: 5 tennis balls.

2. **Tennis balls from the cans**: Each can has 3 tennis balls, and Roger buys 2 cans.  
   So, \( 2 \times 3 = 6 \) tennis balls from the cans.

3. **Total tennis balls Roger has after buying the cans**:  
   \( 5 + 6 = 11 \) tennis balls.

4. **Tennis balls Roger gives to his friend**: He gives away 4 tennis balls.  
   \( 11 - 4 = 7 \).

**Final Answer**: Roger has **7 tennis balls** now.


In [47]:
prompt_fsi_zeroshot_cot= """Q: Let’s analyze the financial health of Company X. The company has a revenue of $10 million, expenses of $7 million, and a debt of $2 million. Calculate the net profit and the debt-to-equity ratio.
A: The answer is"""

In [48]:
response = client.chat.completions.create(
    model="gpt-35-turbo",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_fsi_zeroshot_cot}
    ],
    temperature=0,
    max_tokens=100,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


To calculate the net profit, you subtract the total expenses from the total revenue:

Net Profit = Revenue - Expenses
Net Profit = $10 million - $7 million
Net Profit = $3 million

Therefore, the net profit of Company X is $3 million.

To calculate the debt-to-equity ratio, you divide the total debt by the total equity:

Debt-to-Equity Ratio = Total Debt / Total Equity
Debt-to-Equity Ratio = $2 million / ($


In [49]:
prompt_fsi_zeroshot_cot= """Q: Let’s analyze the financial health of Company X. The company has a revenue of $10 million, expenses of $7 million, and a debt of $2 million. Calculate the net profit and the debt-to-equity ratio.
A: Let's think step by step."""

In [50]:
response = client.chat.completions.create(
    model="gpt-35-turbo",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_fsi_zeroshot_cot}
    ],
    temperature=0,
    max_tokens=1000,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


Sure, let's break it down step by step.

1. Net Profit:
Net Profit = Revenue - Expenses
Net Profit = $10 million - $7 million
Net Profit = $3 million

2. Debt-to-Equity Ratio:
Debt-to-Equity Ratio = Total Debt / Total Equity
Total Equity = Total Assets - Total Debt
Total Assets = Revenue
Total Debt = $2 million

Total Equity = $10 million - $2 million
Total Equity = $8 million

Debt-to-Equity Ratio = $2 million / $8 million
Debt-to-Equity Ratio = 0.25

Therefore, the net profit for Company X is $3 million and the debt-to-equity ratio is 0.25.


In [139]:
PROMPT_FEW_SHOT_CoT = """
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A: Elif had £10 at the beginning. When she consumed £2, 10-2=8 , £8 remains.
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A:
"""

In [140]:
response = client.chat.completions.create(
    model="gpt-35-turbo",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": PROMPT_FEW_SHOT_CoT}
    ],
    temperature=0,
    max_tokens=1000,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


It seems like Roger started with 5 tennis balls, then bought 2 cans of tennis balls with 3 balls each, totaling 6 more balls. After giving 4 balls to his friend, he would have 5 + 6 - 4 = 7 tennis balls left.


** Auto-COT ** uses zero-shot-cot results just like few-shot learning for reasoning. Instead of using few-shot-cot, auto-cot can be useful and easy because you don't need to create manual examples (labels/reasonings)

In [97]:
prompt_auto_cot = """
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Lets think step by step.Roger had 5 tennis balls at the beginning. He bought 2 cans of tennis balls, each with 3 balls, so he now has 5+2x3=11 tennis balls. After giving 4 to his friend, he has 11-4=7 tennis balls remaining.
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A:
"""

In [98]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_auto_cot}
    ],
    temperature=0.5,
    max_tokens=1000
)

# Access the message content safely
message_content = response.choices[0].message.content
if message_content:
    print(message_content.strip())
else:
    print("The response content is empty.")


The response content is empty.


## 9. Self Consistency

Self-consistency aims "to replace the naive greedy decoding used in chain-of-thought prompting". The idea is to sample multiple, diverse reasoning paths through few-shot CoT, and use the generations to ** select the most consistent answer.**

In the chat scenarios, **Asking the model to self-verify** its own responses. Like a student double-checking their answers, the AI model cross-references its responses to maintain consistency. 

In [71]:
prompt= f"""When I was 6, my sister was half my age. Now
I am 70 how old is my sister?"""

In [72]:
response = client.chat.completions.create(
    model="gpt-35-turbo",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ],
    temperature=0.5,
    max_tokens=20,
    stop=["\nA:"]
)

# Print the result
print(response.choices[0].message.content.strip())


If your sister was half your age when you were 6, that means she was 3 years


In [73]:
prompt2= f"""When I was 6, my sister was half my age. Now
I am 70 how old is my sister? Let's think step by step"""

In [75]:
response = client.chat.completions.create(
    model="gpt-35-turbo",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt2}
    ],
    temperature=0,
    max_tokens=1000
)

# Print the result
print(response.choices[0].message.content.strip())


Sure, let's break it down step by step:

1. When you were 6, your sister was half your age, which means she was 3 years old at that time (half of 6).

2. The age difference between you and your sister is 3 years (6 - 3 = 3).

3. Now that you are 70 years old, the age difference between you and your sister remains the same, which is 3 years.

4. To find out your sister's current age, you can add the age difference to your current age. So, 70 + 3 = 73.

Therefore, your sister is 73 years old now.


In [76]:
prompt3=f"""
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5
Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:"""

In [81]:
from openai import OpenAI

def call_openai(num_times, start_phrase, temperature):
    # Initialize the OpenAI client
    #client = OpenAI(api_key='your-api-key')  # Replace with your actual API key

    for i in range(num_times):
        # Define the messages for the chat completion
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": start_phrase}
        ]

        # Send a chat completion request
        response = client.chat.completions.create(
            model="gpt-35-turbo",  # Use the appropriate model name
            messages=messages,
            temperature=temperature,
            max_tokens=100
        )

        # Extract and print the assistant's reply
        reply = response.choices[0].message.content.strip()
        print(f"Response {i + 1}:\n{reply}")
        print("*****************************")


In [82]:
call_openai(10, prompt3, temperature = 1)

Response 1:
If you were 6 and your sister was half your age, your sister was 3 at that time. The age gap between you and your sister is constant, so when you are 70, your sister would be 70 - (6 - 3) = 67 years old.
*****************************
Response 2:
If you were 6 and your sister was half your age, that means she was 3 years old at that time. The age gap between you and your sister is 3 years. Fast forward to now, as you are 70 years old, your sister would be 70 - 3 = 67 years old.
*****************************
Response 3:
If you were 6 and your sister was half your age, she was 3 years old at that time. The age gap between you two is 3 years. So, as you are now 70 years old, your sister would be 70 - 3 = 67 years old.
*****************************
Response 4:
If you were 6 years old and your sister was half your age, that means your sister was 3 years old at that time. Now, if you are 70 years old, your sister is 70 - (6 - 3) = 67 years old.
*****************************
Respon

# 9. Step-Back Prompting Technique

You start by providing the model with a prompt or question.
The model generates a response based on the initial prompt.
Instead of immediately accepting the response, you prompt the model to review or analyse its own response. This could involve asking the model to check for errors, verify facts, or consider alternative approaches.
Based on the reassessment, the model generates a refined or corrected response.

In [83]:
# function to generate a response based on a prompt  
def generate_response(prompt):  
    messages = [  
        {"role": "system", "content": "You are a helpful assistant."},  
        {"role": "user", "content": prompt}  
    ]  
    response = client.chat.completions.create(  
        model="gpt-4o",  
        messages=messages  
    )  
    return response.choices[0].message.content.strip()  

# function to rephrase the input text  
def rephrase_input(input_text):  
    rephrase_prompt = (  
        "You are a helpful assistant. Please rephrase the following request to make it clearer and a more general question: "  
        f"'{input_text}'"  
    )  
    messages = [  
        {"role": "system", "content": "You are a helpful assistant."},  
        {"role": "user", "content": rephrase_prompt}  
    ]  
    response = client.chat.completions.create(  
        model="gpt-4o",  
        messages=messages  
    )  
    return response.choices[0].message.content.strip()  
  
  
# function to reword and check the answer  
def reword_and_check_answer(original_answer):  
    rewording_prompt = f"Reword and verify the following statement: '{original_answer}'"  
    messages = [  
        {"role": "system", "content": "You are a helpful assistant."},  
        {"role": "user", "content": rewording_prompt}  
    ]  
    response = client.chat.completions.create(  
        model="gpt-4o",  
        messages=messages  
    )  
    return response.choices[0].message.content.strip()

In [84]:
initial_prompt = "Iceland Foods Limited, trading as Iceland, is a British supermarket chain."  
  
# Generate a response based on the initial prompt  
response = generate_response(initial_prompt)  
print("*** Initial Response:\n", response)  
  
# Reword and verify the answer  
checked_response = reword_and_check_answer(response)  
print("*** Reworded and Checked Answer:\n", checked_response)  
  
print("*** Final Response:\n", checked_response)  
  
# if __name__ == "__main__":  
#     initial_prompt = input("Enter your question: ")  
#     response = generate_response(initial_prompt)  
#     print("Initial Response:\n", response)  
#     checked_response = reword_and_check_answer(response)  
#     print("Reworded and Checked Answer:\n", checked_response)  

*** Initial Response:
 That's correct! Iceland Foods Limited, commonly known as **Iceland**, is a British supermarket chain that specializes in frozen foods, including prepared meals, vegetables, and desserts. The retailer also offers non-frozen products, such as fresh produce, dairy, pantry staples, and beverages. It was founded in **1970** by **Malcolm Walker** and another business partner in Shropshire, England.

Iceland is known for its value-oriented approach, targeting budget-conscious customers. Over the years, the chain has become one of the go-to supermarkets for frozen goods in the UK. It has also gained attention for its environmental and ethical initiatives, such as reducing plastic packaging and promoting sustainable fish sourcing. Iceland operates stores across the UK and internationally in some locations as well.

Let me know if you'd like more details! 😊
*** Reworded and Checked Answer:
 Here’s a reworded version of your statement, along with confirmation of its accurac

## 10. Iterative approach

Prompt engineering is an iterative process. If you're unsatisfied with the AI's response, refine your prompt and try again. Analyze the results you receive and consider adjusting your prompt's context, clarity, or structure. This process of trial and error will help you better understand how the AI model interprets your prompts and allow you to fine-tune your approach.

·        Try different prompts to find what works best

·        When attempting few-shot learning, try also to include direct instructions

·        Rephrase a direct instruction set to be more or less concise, e.g.: taking a previous example and giving the next instruction without having to repeat the input

·        Try different personas keywords to see how it affects the response style

·        Use fewer or more examples in the few-shot learning

·        Co-create with AI: An example of a very useful prompt to get a good output from the LLM :

In [85]:
text=f"""
Overall Results
Year Ended December 31, 2023 versus 2022:
Net income attributable to  ordinary shareholders was $1.1 billion for the year ended December 31, 2023, 
which compares to a net loss of $906 million from 2022, as a result of:
• Favorable total investment returns recognized in net income of $1.1 billion for the year ended December 31, 
2023, consisting of the aggregate of net investment income, net realized (losses) gains, net unrealized gains 
(losses) and income (losses) from equity method investments, in comparison to negative total investment 
returns included in net income of $1.2 billion for the year ended December 31, 2022. The variance in total 
investment returns recognized in net income was driven by: 
◦ Net unrealized gains on our other investments, including equities of $397 million, in comparison to net 
unrealized losses in 2022 of $433 million, as a result of strong global equity market performance, 
particularly in the first and fourth quarters of 2023, and tightening high yield credit spreads, in comparison to 
the challenging market environment for the year ended December 31, 2022;
◦ Net realized and unrealized gains on our fixed maturities of $66 million in 2023, compared to net realized 
and unrealized losses of $1.2 billion in 2022, primarily due to a decrease in interest rates across U.S., U.K. 
and European markets in 2023 as compared to significant increases in interest rates in 2022; 
◦ An increase in net investment income of $192 million in 2023 when compared to 2022, consistent with the 
increasing investment income we have earned on a sequential quarterly basis, primarily due to the 
reinvestment of fixed maturities at higher yields, deployment of consideration received from LPT and 
insurance transactions closed over the past 12 months and the impact of rising interest rates on our fixed 
maturities securities that are subject to floating interest rates; and
◦ Income from equity method investments of $13 million, driven by income from our investments in Core 
Specialty and Citco, partially offset by losses from our investment in Monument Re, compared to losses of 
$74 million in 2022, primarily driven by losses from our investment in Monument Re. 
• An increase in other income of $241 million in 2023 when compared to 2022, largely driven by the first quarter 
2023 net gain recognized from the novation of the Enhanzed Re reinsurance of a closed block of life annuity 
policies; and 
• A favorable change in income tax benefit of $238 million, primarily driven by the establishment of a $205 million 
net deferred tax asset related to the enactment of the Bermuda Corporate Income Tax in December 2023. We 
also recorded a $25 million partial release of our deferred tax asset valuation allowance as a result of increases 
in projected taxable income in the U.S. and a reduction in deferred tax assets associated with decreases in 
unrealized losses on investment securities reported in AOCI in the U.S. and U.K. jurisdictions. This was partially 
offset by an increase in the valuation allowance in our U.K. and EU jurisdictions primarily due to losses, 
whereby no corresponding tax benefits were recognized for the period. 
"""

In [86]:
prompt_iterative= f""" Your task is to explain given information in a very simple way.
### Context:
{text}
"""

In [87]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative}
    ]
)

print(response.choices[0].message.content)

Here’s a simple explanation of the information provided:

In 2023, the company made $1.1 billion in profit, while in 2022, it lost $906 million. This big improvement happened because:

1. **Investments Did Better in 2023**:
   - The company made money from investments (like stocks and bonds) instead of losing money like in 2022.
   - Stock (equity) investments did well globally, especially early and late in 2023, leading to $397 million in gains, compared to a loss of $433 million in 2022.
   - The company gained $66 million from bonds in 2023 because interest rates went down, while it lost $1.2 billion in 2022 when interest rates went up.
   - Investment income (money earned from investments like interest and dividends) increased by $192 million. This happened because the company reinvested in higher-yield investments and benefited from higher interest rates on certain securities.

2. **Profits From Partner Investments**: 
   - The company earned $13 million in 2023 from its investmen

**Issue 1:** I want to keep numerical values in more readable output format.

In [88]:
prompt_iterative1= f""" Your task is to organize given information in table format by keeping numarical values. 

### Context:
{text}
###
"""

In [89]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative1}
    ]
)

print(response.choices[0].message.content)

Here is the information organized into a table format:

| **Category**                                                       | **2023**               | **2022**               | **Variance (2023 vs 2022)**                                                                              |
|---------------------------------------------------------------------|------------------------|------------------------|--------------------------------------------------------------------------------------------------------|
| **Net Income**                                                     | $1.1 billion          | $(906) million         | Favorable change of $2.006 billion                                                                     |
| **Total Investment Returns in Net Income**                         | $1.1 billion          | $(1.2) billion         | Increase of $2.3 billion                                                                              |
| **Net Unrealized Gains (Losses) on Oth

**Issue 2:** It is long so I need a brief summary.

In [90]:
prompt_iterative2= f""" 
Your task is to organize given information briefly in table format by keeping numarical values.

Then, provide simple explanation by using at most 20 words.

### Context:
{text}
###
"""

In [91]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative2}
    ]
)

print(response.choices[0].message.content)

### Organized Table:

| Key Metric/Category                     | 2023 Value         | 2022 Value           | Change Explanation                                           |
|-----------------------------------------|---------------------|----------------------|-------------------------------------------------------------|
| **Net Income/(Loss)**                   | $1.1 billion        | ($906 million)       | Improved due to favorable investment returns and tax benefits. |
| **Total Investment Returns**            | $1.1 billion        | ($1.2 billion)       | Shift from losses to gains due to equity and fixed maturity. |
| - Net Unrealized Gains/Losses (Equities)| $397 million        | ($433 million)       | Driven by strong global equity markets and tightening spreads. |
| - Realized/Unrealized Gains (Fixed)     | $66 million         | ($1.2 billion)       | Decrease in interest rates boosted fixed maturity performance. |
| - Net Investment Income                 | $192 million (↑)  

In [92]:
prompt_iterative3= f""" Your task is to organize given information briefly in table format by keeping numarical values.

Then, provide simple explanation of the given context by using at most 20 words.

Format everything as HTML that can be used in a website. 

### Context:
{text}
###
"""

In [93]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative3}
    ]
)

print(response.choices[0].message.content)

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Year-End Results</title>
</head>
<body>
    <h1>Year-End Results: December 31, 2023 vs. December 31, 2022</h1>
    <table border="1" cellpadding="10">
        <thead>
            <tr>
                <th>Category</th>
                <th>2023 ($ Millions)</th>
                <th>2022 ($ Millions)</th>
                <th>Change</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>Net Income (Loss) Attributable to Shareholders</td>
                <td>1,100</td>
                <td>(906)</td>
                <td>+2,006</td>
            </tr>
            <tr>
                <td>Net Investment Returns</td>
                <td>1,100</td>
                <td>(1,200)</td>
                <td>+2,300</td>
            </tr>
            <tr>
                <td>Net Unrealized Gains/Losses on Equi

## Display the HTML content of the completion response

In [94]:
from IPython.display import display, HTML
display(HTML(response.choices[0].message.content))

Category,2023 ($ Millions),2022 ($ Millions),Change
Net Income (Loss) Attributable to Shareholders,"1,100",(906),"+2,006"
Net Investment Returns,"1,100","(1,200)","+2,300"
Net Unrealized Gains/Losses on Equities,397,(433),+830
Net Gains/Losses on Fixed Maturities,66,"(1,200)","+1,266"
Net Investment Income,192,0,+192
Income/Loss from Equity Method Investments,13,(74),+87
Other Income,241,0,+241
Income Tax Benefit,238,0,+238
